# Speech-To-Text Model Fine-Tuning for Various Accents on Engineerning Domains


Preparing Environment

In [ ]:
!pip install -U datasets
#!pip install git+https://github.com/huggingface/transformers
!pip install evaluate>=0.30
!pip install jiwer
!pip install accelerate -U
!pip install wandb
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

In [ ]:
!pip install --upgrade transformers>=4.30.0

In [ ]:
#HuggingFace Login (Optional, only when access to private datasets needed)
from huggingface_hub import notebook_login
notebook_login()

Import FeatureExtractor and Tokenizer

In [ ]:
from tqdm import tqdm
from transformers.models.whisper.feature_extraction_whisper import WhisperFeatureExtractor
# 파인튜닝을 진행하고자 하는 모델의 feature extractor를 로드
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer
# 파인튜닝을 진행하고자 하는 모델의 tokenizer를 로드
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-base", language="English", task="transcribe")

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Combine to WhisperProcessor(Using Tiny Model for now)

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="English", task="transcribe")

Prepare DataSets

In [ ]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from datasets import Audio

In [ ]:
!rm -rf ~/.cache/huggingface/datasets/krishan23___indian_english
#데이터셋 캐시 삭제

In [ ]:
AccentDataset = load_dataset("yongjune2002/MITOCW-whisper")
#3GB에 해당하는 Indian Accent Dataset

README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

(…)-00000-of-00002-d00b4ba211a3c329.parquet:   0%|          | 0.00/429M [00:00<?, ?B/s]

(…)-00001-of-00002-24bfd76ad7724a49.parquet:   0%|          | 0.00/411M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/987 [00:00<?, ? examples/s]

In [ ]:
AccentDataset = AccentDataset.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
def is_under_30s(example):
    array = example["audio"]["array"]
    sr    = example["audio"]["sampling_rate"]
    duration = len(array) / sr
    return duration <= 30.0

AccentDataset = AccentDataset.filter(is_under_30s)

Filter:   0%|          | 0/6765 [00:00<?, ? examples/s]

In [ ]:
print(AccentDataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 987
    })
})


In [ ]:
def prepare_dataset(batch):
    # 오디오 파일을 16kHz로 로드
    audio = batch["audio"]

    # input audio array로부터 log-Mel spectrogram 변환
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # target text를 label ids로 변환
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [ ]:
ProcessedAccentDataset = AccentDataset.map(prepare_dataset, remove_columns=AccentDataset.column_names["train"], num_proc=None)

Map:   0%|          | 0/987 [00:00<?, ? examples/s]

In [ ]:
ProcessedAccentDataset.push_to_hub("yongjune2002/MITOCW-Whisper-Processed")

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yongjune2002/MITOCW-Whisper-Processed/commit/045e8f63dbb495bfae33a3a2e5f3b64a6c53a72a', commit_message='Upload dataset', commit_description='', oid='045e8f63dbb495bfae33a3a2e5f3b64a6c53a72a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yongjune2002/MITOCW-Whisper-Processed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yongjune2002/MITOCW-Whisper-Processed'), pr_revision=None, pr_num=None)

CheckPoint : Data Is Preprocessed and Uploaded to HuggingFace

In [ ]:
ProcessedAccentDataset = load_dataset("yongjune2002/Whisper_IndianAccent")

Generating train split:   0%|          | 0/6765 [00:00<?, ? examples/s]

In [ ]:
print(ProcessedAccentDataset)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 6765
    })
})
